# Question: Does the company have ethical corporate governance principles?

The quick definition of "corporate governance" is "the rules, practices, and processes by which a company is directed and controlled." In less dry terms, issues we look at in the corporate-governance area include the quality and composition of companies' management teams and boards of directors, and how they interact with stakeholders, including investors.

One major area to dig into is executive and CEO pay. Are the CEO and other top executives compensated fairly and according to performance measures? Pay plans that reflect actual performance are a positive mark, and egregiously overpaid CEOs are a negative. ESG investors should also frown on outrageous perks enjoyed by executives on the shareholder dime, like private jets and expensive club memberships.

ESG investors should also examine each company's board of directors. Is the board diverse, composed of folks who are able to push back against management when necessary? Examine aspects like director tenure -- long tenure can lead to a less diverse, robust board. The purpose of the directors is to be a voice for shareholders, so it's not good to see an entrenched or complicit board that will rubber-stamp every whim of management.

It's a good sign when a board of directors has an independent chair. If the CEO also holds the chairperson role, it's much harder for a board to do its job properly. Combining the chairperson and CEO roles can make the leader more like a monarch than a company employee, which is what a CEO actually is.

Another point to consider: Does the company have multiple classes of stock, with one or more classes controlled by executives who have supervoting rights? Supervoting rights mean those shares held by management or insiders can have many more votes than those held by regular shareholders; while a normal shareholder has one vote per share, management-controlled supervoting shares might get 10 votes per share. That would indicate that ordinary shareholders have little voice in votes, a shareholder-unfriendly scenario.

Companies with a collection of poor corporate-governance policies are red-flagged and possibly disqualified from inclusion in our ESG portfolio.

# Data Sources

**Women % Board Representation**

https://2020wob.com/educate2/

# Imports

In [1]:
# All necessary imports
from bs4 import BeautifulSoup
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)

import re
import time

import warnings
warnings.simplefilter("ignore")

# Getting Companies

In [43]:
# Getting site
page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(page.content, 'html.parser')

# Finding table
table = soup.find(id='constituents').tbody

# Getting table rows
table_rows = table.find_all('tr')

# Finding company in row
companies = []
for row in table_rows[1:70]:
    elements = row.find_all('td')
    company = elements[1].text
    companies.append(company)

# Cleaning company rows
cleaned_companies = []
for company in companies: 
    if company == 'Alphabet Inc. (Class A)':
        pass
    elif '(Class B)' in company:
        company = company.replace('(Class B)','')
        cleaned_companies.append(company)
    elif '(Class C)' in company:
        company = company.replace('(Class C)','')
        cleaned_companies.append(company)
    else:
        cleaned_companies.append(company)

# Defining final companies
companies = cleaned_companies

# Getting Data on Women's Board Representation

In [103]:
companies = ['Air Products & Chemicals','Alphabet Inc.','Amazon.com Inc.','American Tower Corp.']

In [104]:
'''
Because of the likelihood and trouble getting blocked by Indeed and 
Comparably, I used randomized delays to decrease the bot-like nature
of the scraping tool.
'''
delays = [5, 2, 4, 7, 8, 9]
delay = np.random.choice(delays)

In [105]:
perc_female_board = pd.DataFrame(columns=['company','percent_female_on_board'])

# Set path to chromedriver
PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
# Define options 
options = Options()
# Remove pop up window
#     options.add_argument("--headless")
# Define driver
driver = webdriver.Chrome(PATH, options=options)
# # Define driver
# driver = webdriver.Chrome(PATH)
driver.set_window_size(1080,800)
# Define url
url= "https://2020wob.com/educate2/"
# Get website
driver.get(url)
time.sleep(5)

for company in companies:
    try:
        # Find search bar
        search_bar = driver.find_element_by_xpath('//*[@id="CName"]')
        # Clear search bar
        search_bar.clear()
        # Enter company name into search bar
        search_bar.send_keys(company)
        # Time delay
        time.sleep(delay)
        try:
            # Click company in dropdown
            driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
        except:
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(company)
            dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
            if dropdown_options == '':
                split_company = company.split()
                # Clear search bar
                search_bar.clear()
                # Enter company name into search bar
                search_bar.send_keys(split_company[0])
                time.sleep(delay)
                dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                if dropdown_options.count('\n') == 0:
                    driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                elif dropdown_options.count('\n') > 0:
                    search_bar.clear()
                    # Enter company name into search bar
                    search_bar.send_keys(split_company[0],' ',split_company[1])
                    time.sleep(delay)
                    dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                    if dropdown_options.count('\n') == 0:
                        driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                    else:
                        pass
        # Time delay
        time.sleep(delay)
        # Enter
        search_bar.send_keys(Keys.ENTER)
        # Time delay
        time.sleep(delay)
        # Get percent women
        percent_women = int((driver.find_element_by_xpath('//*[@id="genderdiversityindex"]/div/div/table/tbody/tr/td[5]').text).replace('%','')) / 100
        # Time delay
        time.sleep(delay)


        perc_female_board = perc_female_board.append({'company':company,
                                                      'percent_female_on_board':percent_women}, ignore_index=True)
        
    except:
        perc_female_board = perc_female_board.append({'company':company,
                                                      'percent_female_on_board':'NaN'}, ignore_index=True)
    
perc_female_board = perc_female_board.sort_values(by='percent_female_on_board', ascending=False)


In [106]:
perc_female_board

,company,percent_female_on_board
2,Amazon.com Inc.,0.50
0,Air Products & Chemicals,0.33
1,Alphabet Inc.,0.27
3,American Tower Corp.,0.27


In [47]:
def get_females_on_board(companies):

    perc_female_board = pd.DataFrame(columns=['company','percent_female_on_board'])

    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    #     options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # # Define driver
    # driver = webdriver.Chrome(PATH)
    driver.set_window_size(1080,800)
    # Define url
    url= "https://2020wob.com/educate2/"
    # Get website
    driver.get(url)
    time.sleep(5)

    for company in companies:
        try:
            # Find search bar
            search_bar = driver.find_element_by_xpath('//*[@id="CName"]')
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(company)
            # Time delay
            time.sleep(delay)
            try:
                # Click company in dropdown
                driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
            except:
                # Clear search bar
                search_bar.clear()
                # Enter company name into search bar
                search_bar.send_keys(company)
                dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                if dropdown_options == '':
                    split_company = company.split()
                    # Clear search bar
                    search_bar.clear()
                    # Enter company name into search bar
                    search_bar.send_keys(split_company[0])
                    time.sleep(delay)
                    dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                    if dropdown_options.count('\n') == 0:
                        driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                    elif dropdown_options.count('\n') > 0:
                        search_bar.clear()
                        # Enter company name into search bar
                        search_bar.send_keys(split_company[0],' ',split_company[1])
                        time.sleep(delay)
                        dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                        if dropdown_options.count('\n') == 0:
                            driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                        else:
                            pass
            # Time delay
            time.sleep(delay)
            # Enter
            search_bar.send_keys(Keys.ENTER)
            # Time delay
            time.sleep(delay)
            # Get percent women
            percent_women = int((driver.find_element_by_xpath('//*[@id="genderdiversityindex"]/div/div/table/tbody/tr/td[5]').text).replace('%','')) / 100
            # Time delay
            time.sleep(delay)


            perc_female_board = perc_female_board.append({'company':company,
                                                          'percent_female_on_board':percent_women}, ignore_index=True)

        except:
            perc_female_board = perc_female_board.append({'company':company,
                                                          'percent_female_on_board':'NaN'}, ignore_index=True)
    
    perc_female_board = perc_female_board.sort_values(by='percent_female_on_board', ascending=False)

    return perc_female_board




In [48]:
get_females_on_board(companies)

,company,percent_female_on_board
0,3M Company,0.36
1,Abbott Laboratories,0.36
2,AbbVie Inc.,0.27
3,Abiomed,0.29
4,Accenture,0.36
5,Activision Blizzard,0.2
6,Adobe Inc.,0.3
7,Advanced Micro Devices,0.25
8,Advance Auto Parts,0.11
9,AES Corp,0.3
